In [16]:
from constants import TIMBRE_GROUP
import h5py
from tracks_to_assignments import get_anns
from constants import valid_data_types

In [6]:
anns = get_anns()

In [8]:
tree = anns['beat_coefs']

In [17]:
track_file = '../tracks.h5'

with h5py.File(segment_file, 'r') as tracks:
    raw_by_dtype = {}
    for grp in tracks.values():
        for dtype in valid_data_types:
            values = grp[dtype].value
            if dtype == TIMBRE_GROUP:
                values = values[:, 1:]
            for x in values:
                y = anns[dtype].tree.get_nns_by_vector(x, 1, include_distances=True)
                print(y)
                break
        break


TypeError: 'numpy.int64' object is not iterable